# Datentransformation Bureau

Ziel dieser Datenaufbereitung ist es alle Variablen so in einen Datensatz zu überführen.

In [1]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from IPython.display import display, Markdown

In [2]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [3]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
bureau = pd.read_csv(DATASET_DIR / "bureau.csv")
bureau_balance = pd.read_csv(DATASET_DIR / "bureau_balance.csv")
des = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [4]:
des.loc[des['Table']=="bureau.csv", "Row":"Special"]

,Row,Description,Special
125,SK_ID_CURR,"ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau",hashed
126,SK_BUREAU_ID,Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application),hashed
127,CREDIT_ACTIVE,Status of the Credit Bureau (CB) reported credits,NaN
128,CREDIT_CURRENCY,Recoded currency of the Credit Bureau credit,recoded
129,DAYS_CREDIT,How many days before current application did client apply for Credit Bureau credit,time only relative to the application
130,CREDIT_DAY_OVERDUE,Number of days past due on CB credit at the time of application for related loan in our sample,NaN
131,DAYS_CREDIT_ENDDATE,Remaining duration of CB credit (in days) at the time of application in Home Credit,time only relative to the application
132,DAYS_ENDDATE_FACT,Days since CB credit ended at the time of application in Home Credit (only for closed credit),time only relative to the application
133,AMT_CREDIT_MAX_OVERDUE,Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample),NaN
134,CNT_CREDIT_PROLONG,How many times was the Credit Bureau credit prolonged,NaN


In [5]:
todo = list(bureau.columns)

#### Keine Datenaufbereitung für Primär- und Fremdschlüssel

In [6]:
keys = ["SK_ID_CURR", "SK_ID_BUREAU"]

In [7]:
todo = [element for element in todo if element not in keys]

#### Datenaufbereitung metrische Daten

In [8]:
m_heads = [element for element in todo if bureau[element].dtype.name != "object"]

##### Gruppierung

In [9]:
quants = [0, 0.25, 0.5, 0.75, 1]

In [10]:
df = bureau[["SK_ID_CURR", m_heads[0]]]
df = df.groupby(by = ["SK_ID_CURR"]).quantile(quants)
df = df.unstack(level=-1)

In [12]:
df.head()

DAYS_CREDIT                               
                  0.00    0.25    0.50    0.75   1.00
SK_ID_CURR                                           
100001         -1572.0  -894.0  -857.0 -439.50  -49.0
100002         -1437.0 -1122.0 -1042.5 -602.75 -103.0
100003         -2586.0 -1873.5 -1205.5 -732.75 -606.0
100004         -1326.0 -1096.5  -867.0 -637.50 -408.0
100005          -373.0  -255.0  -137.0  -99.50  -62.0

In [ ]:
for index, head in enumerate(m_heads):
    
    df = bureau[["SK_ID_CURR", head]]
    df = df.groupby(by = ["SK_ID_CURR"]).quantile(quants)
    df = df.unstack(level=-1)
    
    try:
        mets = mets.join(df , on = "SK_ID_CURR")
    except (ValueError, NameError):
        mets = df

In [ ]:
# Löschen der metrischen Daten aus der TODO Liste
todo = [element for element in todo if element not in m_heads]

In [ ]:
mets.head()

#### Datenaufbereitung kategorische Daten

In [ ]:
cat_heads = [element for element in todo if bureau[element].dtype.name == "object"]

In [ ]:
for index, head in enumerate(cat_heads):
    
    df = bureau[["SK_ID_CURR", head]]
    df = df.groupby(["SK_ID_CURR", head]).size()
    df = df.unstack()
    
    try:
        cats = cats.join(a , on="SK_ID_CURR")
    except (ValueError, NameError):
        cats = df

In [ ]:
# Löschen der metrischen Daten aus der TODO Liste
todo = [element for element in todo if element not in cat_heads]

In [ ]:
cats.head()

#### Zusammenführung metrischer und kategorischer Daten

In [ ]:
bureau = mets.join(cats, on="SK_ID_CURR")

In [ ]:
bureau.head()

### Abspeichern der Daten

In [ ]:
bureau.to_csv(DATASET_DIR / "Datengruppierung" / "bureau.csv")

In [ ]:
todo